In [3]:
%matplotlib inline

import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import random
from collections import Counter, namedtuple
from math import log

In [4]:
fake1 = np.linspace(0, 40, 40, dtype=int)
fake2 = fake1.copy()

replace_at = np.random.randint(0, 40, size=1)
for i in range(0, len(fake1)): # Replacing random elements in the uniform distribution
    if(i in replace_at):
        fake1[i] = np.random.randint(0, 40)
print(fake1)

fake2 = np.linspace(0, 40, 40, dtype=int)
replace_at = np.random.randint(0, 40, size=1)
for i in range(0, len(fake2)): # Replacing random elements in the uniform distribution
    if(i in replace_at):
        fake2[i] = np.random.randint(0, 40)
print(fake2)
# mini_fake1 = random.sample(fake1, 10)
# print(mini_fake1)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 26 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 40]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35  8 37 38 40]


In [5]:
def kl_div_scipy(p,q):
    p = np.asarray(p, dtype=np.float)
    q = np.asarray(q, dtype=np.float)

#     return np.sum(np.where(p != 0, p * np.log(p / q), 0))
    return st.entropy(p,q)

In [6]:
kl_div_scipy(fake1,fake2)

0.034556015664196998

In [7]:
def kl_divergence(p,q):
    """ Returns Kl Divergence of two integer lists. Theory at https://www.countbayesie.com/blog/2017/5/9/kullback-leibler-divergence-explained
    :type p: List[int]
    :type q: List[int]
    :rtype: double
    """
    cf1, l1 = Counter(p), len(p)
    cf2, l2 = Counter(q), len(q)
    
    # Pre-processing for using KL Divergence of Frequency Counters cf1 and cf2
    s = set(cf1.keys())
    s.intersection(cf2.keys()) # Collecting all unique elements in cf1 and cf2

    # Normalizing the series to reflect probabilities of occurence
    for e in cf1:
        cf1[e] = float(cf1[e]/l1)
    for e in cf2:
        cf2[e] = float(cf2[e]/l2)
    print("Kl Div from library func = {0}".format(kl_div_scipy(list(cf1.values()),list(cf2.values()))))
    kl_div = 0.0
    for c in s: # For each unique element in both series
        if cf1[c] != 0 and cf2[c] != 0:
            t = cf1[c] * log(cf1[c]/cf2[c])
#             print("cf1[{0}]={1} cf2[{0}]={2} t={3} k={4}".format(c, cf1[c], cf2[c], t, kl_div))
            kl_div += t 
            
            t2 = cf2[c] * log(cf2[c]/cf1[c])
#             print("cf2[{0}]={1} cf1[{0}]={2} t2={3} k={4}".format(c, cf2[c], cf1[c], t2, kl_div))
            kl_div += t2 
    return kl_div

In [8]:
kl_divergence(fake1, fake2)

Kl Div from library func = 0.017328679513998628


0.034657359027997256

In [12]:
data = fake1
# Testing KL Divergence with a standard distribution
DISTRIBUTIONS = [st.uniform, st.norm, st.invgauss]
for distribution in DISTRIBUTIONS:
    # fit dist to data
    params = distribution.fit(data)
    print("Dist: {0}. Parameters: {1}".format(distribution, params))

# Distribution.fit uses scipy.stats._continuous_distns.py --(uses)--> scipy.stats._distn_infrastructure.rv_continuous.fit() --(uses)--> scipy.optimize.optimize.fmin()

Dist: <scipy.stats._continuous_distns.uniform_gen object at 0x0000016164ABE6D8>. Parameters: (-9.9936295516605124e-05, 40.000116111526083)
Dist: <scipy.stats._continuous_distns.norm_gen object at 0x0000016164A3A5C0>. Parameters: (19.75, 11.622714829161042)
Dist: <scipy.stats._continuous_distns.invgauss_gen object at 0x0000016164A882B0>. Parameters: (0.050102514050857791, -34.492910046934568, 1078.8852347800894)
